# Байесовская оценка А/Б-тестов

*Описана механика А/Б-тестов. Рассмотрены примеры байесовского моделирования. Байесовская оценка применена к сравнению конверсий, средних с помощью центральной предельной теоремы, выручки на пользователя, заказов на платящего, средних чеков.*

&nbsp; &nbsp; *- [А/Б тесты](#А/Б-тесты)*  
&nbsp; &nbsp; *- [Байесовское моделирование](#Байесовское-моделирование)*  
&nbsp; &nbsp; *- [Конверсии](#Конверсии)*   
&nbsp; &nbsp; *- [Средние](#Средние)*    
&nbsp; &nbsp; *- [Выручка на пользователя](#Выручка-на-пользователя)*  
&nbsp; &nbsp; *- [Заказы на платящего](#Заказы-на-платящего)*  
&nbsp; &nbsp; *- [Средний чек](#Средний-чек)*  
&nbsp; &nbsp; *- [Заключение](#Заключение)*  
&nbsp; &nbsp; *- [Ссылки](#Ссылки)*  

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

# А/Б тесты  

В мобильные приложения и веб-сервисы вносят изменения для роста выручки, конверсий, вовлеченности и других ключевых метрик. Повышение цен может снизить конверсию, но увеличить прибыль. Точный эффект непредсказуем - изменения могут ухудшить продукт. По оценкам только треть релизов приводит к положительным результатам [[MicroExp](https://www.microsoft.com/en-us/research/publication/online-experimentation-at-microsoft/)]. Поэтому необходимо измерять эффект от новой функциональности.

<center>
<img src="./figs/experiment_versions_ru.png" alt="experiment_versions" width="400"/>
    
<em>Увеличение стоимости (справа) может привести к росту выручки, но падению конверсии. Эффект непредсказуем и требует измерения. </em>
</center>

Сравнение метрик до и после релиза не всегда позволяет оценить изменение. При "сильном" эффекте будет виден скачок в метрике, но "слабый" эффект может быть незаметен на фоне колебаний. На метрики также влияют изменения в других частях продукта, привлекаемом трафике или общей активности аудитории - например, запуск рекламной акции. Поэтому изменения метрик после релиза не всегда можно объяснять новой функциональностью. 

<center>
<img src="./figs/effect_size.png" alt="effect_size"  width="900"/>
<em>При оценке эффекта сильные изменения заметны в динамике метрики (см. падение на 30%), но слабые изменения могут быть незаметны на фоне колебаний (см. рост на 3%). Кроме того, изменение метрик после релиза не всегда связано с новой функциональностью. </em>
</center>

Для более точной оценки эффекта используют А/Б-тесты. В этом подходе версию без изменений и измененный вариант сервиса запускают параллельно. При попадании на сайт или в приложение пользователь случайным образом определяется в один из вариантов. В каждой группе собирают данные, вычисляют и сравнивают интересующие метрики. Эксперимент останавливают, если одна из групп лидирует или продолжать тест неоправдано. В итоге принимается решение о дальнейших действиях - как правило, о выборе одного из вариантов для всех пользователей. Описание деталей см. также в [[TrustworthyAB]((https://www.amazon.com/gp/product/B0845Y3DJV)].

<center>
<img src="./figs/ab_test.png" alt="ab_test" width="800"/>
    
<em>Схема А/Б эксперимента: тестируемые версии сервиса запускают параллельно, пользователи случайным образом попадают в один из вариантов. В каждой группе вычисляют интересующие метрики, по результатам сравнения определяют дальнейшие действия. </em>
</center>

Причинная диаграмма [[CausalDAG](https://en.wikipedia.org/wiki/Causal_graph)] А/Б-тестов следующая. Метрики определяются действиями пользователей в сервисе. Действия зависят от версии сайта или приложения (например, доступных тарифных планов), внешних факторов (например, сезонности) и также отличаются между сегментами пользователей (новые, постоянные клиенты и др.). В А/Б-тесте версии запускают одновременно и пользователей случайно делят между вариантами. Внешние факторы сохраняются, но при сравнении за одинаковый период их влияние на группы одинаково. При случайном делении пользователей состав сегментов можно считать одинаковым. В итоге разница метрик между группами объясняется функциональностью приложения.  

<center>
<img src="./figs/causal.png" alt="causal" width="600"/>
    
<em>Метрики определяются действиями пользователей в сервисе. Действия зависят от текущей версии сайта или приложения, внешних факторов и также отличаются между сегментами пользователей. Случайное деление пользователей между вариантами А/Б-теста позволяет говорить об одинаковом влиянии сегментов на метрики, а одновременный запуск - об одинаковом влиянии внешних факторов. В итоге разницу метрик между группами можно объяснять тестируемой функциональностью. </em>
</center>

По итогам эксперимента нужно оценить метрики, эффект и выбрать "лучшую" группу. Точные значения метрик неизвестны. Их удобнее рассматривать как случайные величины. Распределения вероятностей этих величин нужно подобрать для наибольшей совместимости с экспериментальными данными. Сравнение распределений позволяет оценить эффект. Для презентации удобна точечная оценка метрик и интервал наибольшей плотности вероятности. Например, среднее значение метрики в группе А $p_A = 7.1 \pm 0.2$, в группе Б $p_B = 7.4 \pm 0.3$. Эффект можно охарактеризовать относительной разностью $(p_B - p_A) / p_A = 4.2 \pm 0.2 \%$. Для выбора "лучшей" группы можно оценить с какой вероятностью метрика в группе Б больше метрики в группе А, например, $P(p_B > p_A) = 95\%$. Вероятность здесь и далее понимается в субъективном смысле - как мера уверенности в определенном исходе процесса с несколькими возможными исходами [[SubjProb](https://en.wikipedia.org/wiki/Probability_interpretations#Subjectivism)].

<center>
<img src="./figs/ab_metric_random.png" alt="ab_metric_random" width="500"/>
    
<em>
В А/Б тесте нужно оценить метрики, эффект и выбрать "лучшую" группу. Неизвестные точные значения метрик удобнее рассматривать как случайные величины. Их распределения вероятностей нужно подобрать для наибольшей совместимости с экспериментальными данными. Сравнение распределений позволяет оценить эффект.
</em>
</center>

Пока данных мало, неопределенность в оценках метрик большая. По мере набора данных оценки уточняются. Вместе с этим растет уверенность, какая из групп лучше. Когда уверенность достигает достаточного значения, эксперимент можно останавливать. Возможны другие критерии остановки.

<center>
<img src="./figs/ab_dynamics.png" alt="ab_dynamics" width="400"/>
<em>
По мере набора данных снижается неопределенность в оценках метрик и растет уверенность, какая из групп лучше. Когда уверенность достигает достаточного значения, эксперимент можно останавливать.
</em>
</center>

Для оценки распределений метрик на основе экспериментальных данных используется байесовское моделирование.

# Байесовское моделирование

Несколько примеров байесовского моделирования.

Вы нашли небольшую коробку в ящике стола. Вы не помните, что внутри и угадываете перед открытием коробки. У вас несколько вариантов - документы, старая электроника, возможно, пусто. Документы вы обычно храните отдельно, также они не влезли бы в коробку. Коробка может быть пустой, но вряд ли вы бы стали хранить пустую коробку. Скорее всего электроника. Вы открываете коробку. В ней сувениры из прошлой поездки.

Рассуждения выше можно сделать количественными. Нужно предположить несколько вариантов содержимого коробки и для каждого оценить вероятность, что предмет в коробке $P(предмет | коробка)$. 
Это делается с помощью соотношения Байеса. Вероятность $P(предмет | коробка)$ пропорциональная вероятности, что вы из всех пришедших в голову вариантов в коробке именно этот предмет $P(предмет)$ и вероятности, что вы положили этот предмет именно в найденную коробку $P(коробка|предмет)$.

$$
P(предмет | коробка) \propto P(коробка|предмет) P(предмет)
$$

Например, $P(документы) = 0.3$, документы хранятся отдельно $P(коробка|документы) = 0.2$. $P(документы | коробка) \propto 0.6$. Электроника может быть $P(электроника) = 0.5$. Также $P(коробка|электроника)$ = 0.6. Для пустой коробки остается $P(пусто) = 0.2$. Пусть $P(коробка|пусто)$ = 0.2. 

$$
P(документы | коробка) \propto P(коробка|документы) P(документы) = 0.3 \cdot 0.2 = 0.06
\\
P(электроника | коробка) \propto P(коробка|электроника) P(электроника) = 0.5 \cdot 0.6 = 0.3 
\\
P(пусто | коробка) \propto P(коробка|пусто) P(пусто) = 0.2 \cdot 0.2 = 0.04
$$

пропуск  
банковская карта  
проездной

На этом примере видны основные элементы байесовского моделирования. Есть данные (или факты). Есть несколько гипотез (или моделей, или предположений), объясняющих данные. Для выбора одной из гипотез нужно оценить, насколько каждая из гипотез согласуется с данными. Насколько хорошо гипотеза объясняет данные $P(данные | модель)$ и уверенность в гипотезе по сравнению с другими $P(модель)$. После чего выбрать наиболее хорошо согласующуюся. Для этого нужно вычислить вероятности получить данные в рамках гипотез (правдоподобия) и априорные вероятности каждой гипотезы. Связь выражается соотношением Байеса:

$$
P(модель | данные) = \frac{ P(данные | модель) P(модель) }{P(данные)} .
$$

$P(модель | данные)$ - апостериорное распределение вероятности, $P(данные | модель)$ - функция правдоподобия, $P(модель)$ - априорное распределение вероятности, $P(данные)$ не имеет специального названия и играет роль нормировочного коэффициента.

Таким образом, в байесовском подходе выбирается набор возможных моделей для объяснения данных. Для каждой модели вычисляется вероятность получить данные в рамках выбранной модели - функция правдоподобия $P(данные|модель)$. Для каждой модели задается уверенность в этой модели относительно остальных - априорная вероятность $P(модель)$. По соотношению Байеса вычисляется апостериорная вероятность $P(модель|данные)$. Выбирается наиболее подходящая модель.

В примере выше в коробке не оказалось ничего из исходных предположений. Это напоминает о том, что ни одна из рассматриваемых гипотез может быть не верна.

<center>
<img src="./figs/bayes_hypotheses.png" alt="bayes_hypotheses" width="600"/>
</center>

Следующий пример. На страницу зашло $N = 1000$ человек, $n_s = 100$ из них нажали кнопку "Продолжить". Как выглядит распределение возможных значений конверсии $\theta$? Вероятность конверсии каждого пользователя можно считать одинаковой, все возможные априорные значения конверсии равновероятными. 

Необходимо оценить вероятность $P(\theta | n_s, N)$. По соотношению Байеса $P(\theta | n_s, N) \propto P(n_s, N | \theta) P(\theta)$. Заход на страницу и клик по кнопке можно представить как случайную величину с 2 исходами с шансом на успех $\theta$. Вероятность $n_s$ конверсий из $N$ с шансом на успех $\theta$ (схема Бернулли [[Bern](https://en.wikipedia.org/wiki/Bernoulli_process)]) задается биномиальным распределением [[BinomDist](https://en.wikipedia.org/wiki/Binomial_distribution)]  $P(n_s, N | \theta) = \mbox{Binom}(n_s, N; \theta)$. Т.к. все возможные априорные значения конверсий равновероятны, априорное распределение равномерно $P(\theta) = \mbox{Unif}(0, 1) = 1$. Распределение $P(\theta | n_s, N)$ будет бета-распределением [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution), [SciPyBeta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html)].


$$
\begin{split}
P(n_s, N | \theta) & = \mbox{Binom}(n_s, N; \theta) = C^{n_s}_{N} \theta^{n_s} (1 - \theta)^{N-n_s}
\\
P(\theta) & = \mbox{Unif}(0, 1) = 1
\\
P(\theta | n_s, N) 
& = \frac{P(n_s, N | \theta) P(\theta)}{P(n_s, N)}
= \frac{P(n_s, N | \theta) P(\theta)}{\int_0^1 d \theta P(n_s, N | \theta) P(\theta)}
\\
& = \frac{\theta^{n_s} (1 - \theta)^{N-n_s}}{\int_0^1 d \theta (1 - \theta)^{N-n_s} \theta^{n_s} }
= \mbox{Beta}(\theta; n_s + 1, N - n_s + 1)
\end{split}
$$

Гипотезы в примере - использование схемы Бернулли для моделирования конверсий и возможные значения $\theta$.

График апостериорного распределения $P(\theta | n_s, N)$ ниже. Мода совпадает со средним в выборке $n_s/N$, наиболее вероятные значения $\theta$ лежат вблизи этого значения.

In [ ]:
ns = 100
ntotal = 1000

p_samp = ns / ntotal
p_dist = stats.beta(a=ns+1, b=ntotal-ns+1)

xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_dist.pdf(x), line_color='black', name='Распределение'))
fig.add_trace(go.Scatter(x=[p_samp, p_samp], y=[0, max(p_dist.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.update_layout(title='$\mbox{Апостериорное распределение} \, P(\\theta | n_s, N)$',
                  xaxis_title='$\mbox{Конверсия} \, \\theta$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()

Еще один пример. На версию А страницы веб-сайта зашло 1000 человек, 100 нажали кнопку "Продолжить". На версию Б зашло 1000 человек, 110 нажали кнопку продолжить. С какой вероятностью конверсия страницы Б выше страницы А?

Нужна вероятность $P(\theta_B > \theta_A)$. Апостериорное распределение каждой группы вычисляется как в предыдущем примере $P(\theta; n_s, N) = \mbox{Beta}(\theta ; n_s + 1, N - n_s + 1)$. Вероятность $P(\theta_B > \theta_A)$ можно оценить аналитически, но чаще это делают численно. Для этого генерируют выборки из апострериорных распределений и сравнивают их между собой. Для заданных параметров $P(\theta_B > \theta_A) = 76.7 \%$.

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

npost = 50000

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_dist_a.pdf(x), line_color='black', name='А'))
fig.add_trace(go.Scatter(x=x, y=p_dist_b.pdf(x), line_color='black', opacity=0.3, name='Б'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='$Конверсия$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()

samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
p_b_gt_a = np.sum(samp_b > samp_a) / npost

print(f"P(theta_b > theta_a): {p_b_gt_a}")

# Конверсии

Конверсии - есть $N$ пользователей, $n_s$ из них сделали целевое действие (сделали заказ, зашли на страницу и т.п.). Всего пользователей $N$, $n_s$ из них сделали целевое действие. Пользователи независимы друг от друга (т.е. действия одного не влияют на других). Конверсию каждого пользователя одинаковы. В этом случае можно моделировать биномиальным распределением. Функция правдоподобия задается биномиальным распределением $P(n_s, N | p) \sim \mbox{Binom}(p, N)$. Априорное распределение конверсий удобно задать бета-распределением $P(p) = \mbox{Beta}(p; \alpha, \beta)$. Зависимость от $p$ без учета нормировочных коэффициентов $\mbox{Beta}(p; \alpha, \beta) \propto p^{\alpha-1}(1-p)^{\beta-1}$. Эта зависимость сохранится для произведения правдоподобия на априорное распределение $P(p | n_s, N) \propto \mbox{Binom}(p, N) \mbox{Beta}(p; \alpha, \beta) \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}$. Априорные распределения с таким свойством называют сопряженными априорными распределениями. При $\alpha=1, \beta=1$ бета-распределение совпадает с однородным - можно выбирать эти значения как стартовые. Либо можно подобрать на основе исторических данных, чтобы априорное распределение конверсий совпадало с историческим значением.

$$
P(model | data) \propto P(data | model) P(model)
$$

$$
P(data | model) = P(n_s, N | p) = \mbox{Binom}(p, N) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(model) = P(p) = \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
\begin{split}
P(model | data) & = P(p | n_s, N)
\\
& \propto \mbox{Binom}(p, N) \mbox{Beta}(p; \alpha, \beta)
\\
& \propto C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
\\
& \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}
\\
& = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$

Примеры бета-распределения на графике ниже [[BetaDist](https://en.wikipedia.org/wiki/Beta_distribution), [SciPyBeta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html)]. При $\alpha = 1, \beta=1$ распределение однородное. Максимум в точке $p = (\alpha-1) / (\alpha + \beta - 2)$. При увеличении $\alpha$ и $\beta$ распределение сужается и приближается к нормальному.  

In [ ]:
x = np.linspace(0, 1, 1000)
fig = go.Figure()
a, b = 1, 1
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='dash',
                             name=f'a={a}, b={b}'))
a, b = 1, 5
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}'))
a, b = 3, 5
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}'))
a, b = 25, 30
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}'))
a, b = 150, 50
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x, a=a, b=b), 
                             mode='lines', line_color='black', line_dash='solid',
                             name=f'a={a}, b={b}')) 
fig.add_trace(go.Scatter(
    x=[0.98, 0.08, 0.32, 0.55, 0.87],
    y=[1.35, 5.00, 2.80, 6.20, 12.0],
    mode="text",
    name=None,
    showlegend=False,
    text=["a=1, b=1", "a=1, b=5", "a=3, b=5", "a=25, b=30", "a=150, b=50"],
    textposition="top left"
))
fig.update_layout(title='Бета-распределение Beta(a, b)',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  showlegend=False,
                  xaxis_range=[0, 1],
                  height=550)
fig.show()

В примере ниже строится оценка неизвестного значения конверсии по данным. Задается точное значение конверсии `p`, генерируется `nsample` точек. По выборке строится апостериорное распределение возможных значений конверсий. Мода апостериорного распределения совпадает со средним в выборке, но не всегда совпадает с точным значением. 

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + (ntotal - ns) 
    return stats.beta(a=a, b=b)
    
def posterior_binom_approx_95pdi(post_dist):
    lower = post_dist.ppf(0.025)
    upper = post_dist.ppf(0.975)
    return lower, upper

def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

p = 0.1
nsample = 1000

exact_dist = stats.bernoulli(p=p)
data = exact_dist.rvs(nsample)

post_dist = posterior_dist_binom(ns=np.sum(data), ntotal=len(data))

x = np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_dist.pdf(x), line_color='black', name='Апостериорное'))
fig.add_trace(go.Scatter(x=[np.sum(data)/len(data), np.sum(data)/len(data)], y=[0, max(post_dist.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_dist.pdf(x))*1.05], 
                         line_color='red', mode='lines', line_dash='dash', name='Точное p'))
fig.update_layout(title='Апостериорное распределение',
                  xaxis_title='p',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, 1],
                  hovermode="x",
                  height=500)
fig.show()

Сравнение двух групп. Выбирается $p_A$, $p_B$ на 5% больше. Для каждой группы генерируется `nsample` точек, на их основе строятся апостериорные распределения конверсий в каждой группе. Вычисляется вероятность конверсия в группе Б больше $P(p_B > p_A)$. 

In [ ]:
p_A = 0.1
p_B = p_A * 1.05
nsample = 1000

exact_dist_A = stats.bernoulli(p=p_A)
exact_dist_B = stats.bernoulli(p=p_B)
data_A = exact_dist_A.rvs(nsample)
data_B = exact_dist_B.rvs(nsample)

post_dist_A = posterior_dist_binom(ns=np.sum(data_A), ntotal=len(data_A))
post_dist_B = posterior_dist_binom(ns=np.sum(data_B), ntotal=len(data_B))

x = np.linspace(0, 1, 1000)
fig = go.Figure()
#fig.add_vline(x=exact_dist.mean(), line_dash='dash', name='Exact')
fig.add_trace(go.Scatter(x=x, y=post_dist_A.pdf(x), line_color='black', name='A'))
fig.add_trace(go.Scatter(x=x, y=post_dist_B.pdf(x), line_color='black', opacity=0.2, name='B'))
fig.add_trace(go.Scatter(x=[exact_dist_A.mean(), exact_dist_A.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
                         mode='lines', line_dash='dash', line_color='black', name='Точное A'))
fig.add_trace(go.Scatter(x=[exact_dist_B.mean(), exact_dist_B.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
                         mode='lines', line_dash='dash', line_color='black', opacity=0.2, name='Точное B'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='p',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[p_A/2, p_A*2],
                  hovermode="x",
                  height=500)
fig.show()

print(f'P(pB > pA): {prob_pb_gt_pa(post_dist_A, post_dist_B)}')

Динамика по мере набора данных. Две группы. Задаются $p_A$, $p_B$ на 5% лучше. В каждой группе генерируется `nstep` точек `nsize` раз (всего `nstep * nsize` точек). По накопленным данным на каждом шаге строятся апостериорные распределения. Строится оценка интервала наибольшей плотности вероятности. Строится вероятность $P(p_B > p_A)$. По мере набора данных интервалы сужаются, вероятность стремится к 1 в пользу лучшей группы.

In [ ]:
pa = 0.1
pb = pa * 1.05

nstep = 1000
size = 150
sa = stats.binom.rvs(p=pa, n=nstep, size=size)
sb = stats.binom.rvs(p=pb, n=nstep, size=size)

df = pd.DataFrame()
df['n_step'] = [nstep] * size
df['sa_step'] = sa
df['sb_step'] = sb
df['N'] = df['n_step'].cumsum()
df['sa'] = df['sa_step'].cumsum()
df['sb'] = df['sb_step'].cumsum()
df['pa'] = df.apply(lambda r: posterior_dist_binom(r['sa'], r['N']).mean(), axis=1)
df[['pa_lower', 'pa_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sa'], r['N'])), axis=1, result_type="expand")
df['pb'] = df.apply(lambda r: posterior_dist_binom(r['sb'], r['N']).mean(), axis=1)
df[['pb_lower', 'pb_upper']] = df.apply(lambda r: posterior_binom_approx_95pdi(posterior_dist_binom(r['sb'], r['N'])), axis=1, result_type="expand")
df['pb_gt_pa'] = df.apply(lambda r: prob_pb_gt_pa(posterior_dist_binom(r['sa'], r['N']), posterior_dist_binom(r['sb'], r['N']), post_samp=10_000), axis=1)


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pa'], name='A',
                         line_color='black'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pa_upper']) + list(reversed(df['pa_lower'])),
                         fill="toself", name='A, 95% HPDI', marker_color='black', opacity=0.2))
fig.add_trace(go.Scatter(x=df['N'], y=df['pb'], name='B',
                         line_color='blue'))
fig.add_trace(go.Scatter(x=list(df['N']) + list(reversed(df['N'])), 
                         y=list(df['pb_upper']) + list(reversed(df['pb_lower'])),
                         fill="toself", name='B, 95% HPDI', marker_color='blue', opacity=0.2))
fig.update_layout(title='$p_A, p_B$',
                  yaxis_tickformat = ',.1%',
                  xaxis_title='N')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['pb_gt_pa'], name='P(pb > pa)',
                         line_color='black'))
fig.update_layout(title='$P(p_B > p_A)$',
                  yaxis_range=[0, 1],
                  xaxis_title='N')
fig.show()

Доля правильно угаданных вариантов. В группе А выбирается значение конверсии `p`. Для группы Б генерируется `nexps` значений в диапазоне +-5% от `p`. Для каждой пары генерируются данные с шагом `n_samp_step`. Считаются апостериорные распределения и $P(p_B > p_A)$. Эксперимент останавливается, если  $P(p_B > p_A)$ достигает `prob_stop`. Или если достигнуто максимальное количество точек `n_samp_max`. Считается доля правильно угаданных групп во всех экспериментах. В данном случае в `nexps = 100` правильно угадано 94. Точность 0.94 близка к `prob_stop = 0.95`. 

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

p = 0.1
nexps = 100
cmp['A'] = [p] * nexps
cmp['B'] = p * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B'] > r['A'] else 'A', axis=1)

prob_stop = 0.95
for i in range(nexps):
    pA = cmp.at[i, 'A']
    pB = cmp.at[i, 'B']
    exact_dist_A = stats.bernoulli(p=pA)
    exact_dist_B = stats.bernoulli(p=pB)
    n_samp_max = 10_000_000
    n_samp_total = 0
    n_samp_step = 10_000
    ns_A = 0
    ns_B = 0
    while n_samp_total < n_samp_max:
        dA = exact_dist_A.rvs(n_samp_step)
        dB = exact_dist_B.rvs(n_samp_step)
        n_samp_total += n_samp_step
        ns_A = ns_A + np.sum(dA)
        ns_B = ns_B + np.sum(dB)
        post_dist_A = posterior_dist_binom(ns=ns_A, ntotal=n_samp_total)
        post_dist_B = posterior_dist_binom(ns=ns_B, ntotal=n_samp_total)
        pb_gt_pa = prob_pb_gt_pa(post_dist_A, post_dist_B)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_dist_A.mean()
            cmp.at[i, 'B_exp'] = post_dist_B.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: {n_samp_total}, {best_gr}, {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

# Средние

Байесовский подход [[SGBS](https://www.amazon.co.uk/Students-Guide-Bayesian-Statistics/dp/1473916364), [SR](https://www.amazon.co.uk/Statistical-Rethinking-Bayesian-Examples-Chapman/dp/036713991X/ref=sr_1_1)] требует построения моделей распределений сравниваемых величин. Выбор модели всегда произвольный и всегда остаются вопросы обоснования модели.

Для многих величин не нужно знать все распределение. Можно ограничиться сравнением средних:
средней выручкой на пользователя, средней длительностью просмотра и т.д. Для средних значений часто применима центральная предельная теорема. Можно приближенно считать, что средние значения в выборках из распределения будут распределены нормально. Причем это не зависит от формы исходного распределения. Центральная предельная теорема позволяет при сравнении средних использовать нормальное распределение в качестве функции правдоподобия.

Есть несколько центральных предельных теорем [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)].
Одна из формулировок следующая. Пусть есть последовательность независимых одинаково распределенных случайных величин $X_1, X_2, \dots$ с конечными математическим ожиданием $\mu$ и дисперсией $\sigma^2$. Пусть $\bar{X}_N = \frac{1}{N} \sum_{i=1}^{N} X_i$ их выборочное среднее. Тогда при стремящемся к бесконечности $N$ распределение центрированных и масштабированных выборочных средних сходится к нормальному распределению [[NormDist](https://en.wikipedia.org/wiki/Normal_distribution), [SciPyNorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html?highlight=norm)] со средним значением 0 и дисперсией 1. Сходимость понимается как сходимость по распределению [[RandVarsConv](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution)].

$$
P \left( \frac{\bar{X}_N - \mu}{\sigma / \sqrt{N}} = x \right) \to Norm(x; 0, 1), \quad N \to \infty
\\
Norm(x ; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\tfrac{(x-\mu)^2}{2 \sigma^2} }
$$

Неформально этот результат можно применить следующим образом. Если взять произвольное распределение со средним значением $\mu$ и диспресий $\sigma^2$, выбирать из него сэмплы длины $N$ и считать среднее в каждом сэмпле, то средние значения сэмплов будут распределены приблизительно нормально $Norm(\mu, \sigma^2/N)$.

<center>
<img src="./figs/central_limit_theorem_ru.png" alt="Центральная предельная теорема" width="800"/>
</center>

Центральная предельная теорема говорит о сходимости к нормальному распределению центрированных и масштабированных выборочных средних $\bar{X}_N$ при стремлении $N$ к бесконечности. Для фиксированного конечного числа $N$ нормальное распределение не гарантируется. При этом есть теоремы, дающие оценку отличия распределения суммы конечного количества случайных величин от нормального - см. [[BerryEsseenTheorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]. Отличие зависит как от количества слагаемых, так и от параметров распределения. 

In [ ]:
a = 1
sample_len = 100
n_samples = 1000

exact_dist = stats.gamma(a=a)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=np.concatenate(samp), histnorm='probability density', name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
#fig.add_vline(exact_dist.mean(), name='Точное среднее')
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=clt_stdev), 
                         mode='lines', line_color='black', line_dash='dash', name='ЦПТ-распределение'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Выборочные средние', nbinsx=50,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Выборочные средние',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

В приведенной формулировке центральная предельная теорема требует существования конечных среднего и дисперсии у исходного распределения. Примерами распределений, для которых эти свойства могут не выполнятся, являются распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] и близкое к нему распределение Ломакса [[LomaxDist](https://en.wikipedia.org/wiki/Lomax_distribution), [SciPyLomax](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lomax.html)]. Плотность вероятности последнего имеет вид

$$
P(x; c) = \frac{c}{(1 + x )^{c + 1}}, \quad x \ge 0, c > 0.
$$

При значениях параметра $c$ меньше или равном 2 дисперсия распределения Ломакса не является конечной. 
На графиках ниже приведена гистограмма средних в сэмплах и нормальное распределение с параметрами, равными среднему и дисперсии средних сэмплов.

На практике распределения ограничены. Поэтому дисперсии и средние конечные. Однако, выборки могут быть сильно перекошены. Для выполнения ЦПТ может требоваться большое количество точек.

In [ ]:
c = 1.7
sample_len = 500
n_samples = 1000

exact_dist = stats.lomax(c=c)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

xaxis_max=10
x = np.linspace(0, xaxis_max, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=np.concatenate(samp)[np.concatenate(samp) < xaxis_max], histnorm='probability density', 
                           name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
#fig.add_vline(exact_dist.mean(), name='Точное среднее')
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=means_stdev), 
                         mode='lines', line_color='black', line_dash='dash', name='ЦПТ-подобное распределение'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Выборочные средние', nbinsx=150,
                          marker_color='green', opacity=0.5))
fig.update_layout(title='Выборочные средние',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  xaxis_range=[0, xaxis_max],
                  height=550)
fig.show()

Сопряженное априорное распределение для нормальной функции правдоподобия.  
Один параметр: $\mu$ меняется, $\sigma$ фиксировано.  
Модель с меняющимися $\mu$ и $\sigma$ см. отдельно.  
Сопряженное априорное распределение - нормальное распределение.

$$
P(data | model) = Norm(x | \mu, \sigma_x^2) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\tfrac{(x - \mu)^2}{2 \sigma_x^2}}
$$

$$
P(model) = Norm(\mu | \mu_0, \sigma_0^2) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
\prod_i^N
Norm(x_i | \mu, \sigma_x^2)
Norm(\mu | \mu_0, \sigma_0^2)
\\
& \propto_{\mu}
\prod_i^N
e^{-\tfrac{(x_i - \mu)^2}{2 \sigma_x^2}}
e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_0^2}} 
\\
& \propto_{\mu}
e^{-\mu^2 \left[\tfrac{N}{2 \sigma_x^2} + \tfrac{1}{2 \sigma_0^2} \right] + 
   2\mu \left[\tfrac{\mu_0}{2 \sigma_0^2} + \tfrac{1}{2 \sigma_x^2} \sum_i^N x_i \right]}
\\
& \propto_{\mu}
e^{-\tfrac{(\mu - \mu_N)^2}{2 \sigma_N^2}}
= Norm(\mu | \mu_N, \sigma_N^2),
\quad
\sigma_N^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_x^2 + N \sigma_0^2},
\quad
\mu_N = \mu_0 \frac{\sigma_N^2}{\sigma_0^2} + \frac{\sigma_N^2}{\sigma_x^2} \sum_i^N x_i
\end{split}
$$

Проверка оцеки параметров нормального распределения

In [ ]:
ConjugateNormalParams = namedtuple('ConjugateNormalParams', 'mu sigma sx')

def initial_params_normal(mu, sigma, sx):
    return ConjugateNormalParams(mu=mu, sigma=sigma, sx=sx)

def posterior_params_normal(data, initial_pars):
    N = len(data)
    sigma_n_2 = (initial_pars.sigma**2 * initial_pars.sx**2) / (initial_pars.sx**2 + N * initial_pars.sigma**2)
    mu_n = initial_pars.mu * sigma_n_2 / initial_pars.sigma**2 + np.sum(data) * sigma_n_2 / initial_pars.sx**2    
    return ConjugateNormalParams(mu=mu_n, sigma=np.sqrt(sigma_n_2), sx=initial_pars.sx)

def posterior_mu_dist(params):
    return stats.norm(loc=params.mu, scale=params.sigma)

def posterior_rvs(params, nsamp):
    mus = stats.norm.rvs(loc=params.mu, scale=params.sigma, size=nsamp)
    return stats.norm.rvs(loc=mus, scale=params.sx, size=nsamp)
    
# def posterior_binom_approx_95pdi(post_dist):
#     lower = post_dist.ppf(0.025)
#     upper = post_dist.ppf(0.975)
#     return lower, upper

# def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
#     sa = post_dist_A.rvs(size=post_samp)
#     sb = post_dist_B.rvs(size=post_samp)
#     b_gt_a = np.sum(sb > sa)
#     return b_gt_a / post_samp

mu = 3
sigma = 1
nsample = 1000

exact_dist = stats.norm(loc=mu, scale=sigma)
data = exact_dist.rvs(nsample)

# todo: avoid setting from data
sx = np.std(data)
mu0 = data[0]
sigma0 = sx

pars = initial_params_normal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_normal(data[1:], pars)

post_mu = posterior_mu_dist(pars)

npostsamp = 10000
post_samp = posterior_rvs(pars, npostsamp)

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_color='black', name='Точное'))
fig.add_trace(go.Scatter(x=x, y=post_mu.pdf(x), line_color='blue', name='$\mbox{Оценка }\mu$'))
fig.add_trace(go.Scatter(x=[np.sum(data)/len(data), np.sum(data)/len(data)], y=[0, max(post_mu.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_mu.pdf(x))*1.05], 
                         line_color='red', mode='lines', line_dash='dash', name='Точное среднее'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='Апострериорное', nbinsx=100,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Апостериорное распределение',
                  xaxis_title='$\mu$',
                  yaxis_title='Плотность вероятности',
                  #xaxis_range=[0, 10],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()

ЦПТ

In [ ]:
def reshape_and_compute_means(sample, n_split):
    n_means = len(sample) // n_split
    samp_reshaped = np.reshape(sample[0 : n_means * n_split], (n_means, n_split))
    means = np.array([x.mean() for x in samp_reshaped])
    return means

def exact_clt_dist(exact_dist, n_split):
    clt_mu = exact_dist.mean()
    clt_stdev = exact_dist.std() / np.sqrt(n_split)
    return stats.norm(loc=clt_mu, scale=clt_stdev)

def sample_clt_dist(means):
    clt_mu = means.mean()
    clt_std = means.std()
    return stats.norm(loc=clt_mu, scale=clt_std)

nsample = 30000
npostsamp = 50000

a = 4
b = 2
exact_dist = stats.gamma(a=a, scale=1/b)
data = exact_dist.rvs(nsample)

nsplit = 30
means = reshape_and_compute_means(data, nsplit)
clt_dist_exact = exact_clt_dist(exact_dist, nsplit)
#clt_dist_samp = sample_clt_dist(means)


x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Выборка', nbinsx=500,
                           marker_color='black', opacity=0.3))
#fig.add_vline(exact_dist.mean(), name='Точное среднее')
fig.add_trace(go.Scatter(x=x, y=clt_dist_exact.pdf(x), 
                         mode='lines', line_color='black', line_dash='dash', name='ЦПТ-распределение'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Выборочные средние', nbinsx=50,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Выборочные средние',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()



# todo: avoid setting from data
sx = np.std(means)
mu0 = means[0]
sigma0 = sx
pars = initial_params_normal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_normal(means[1:], pars)
post_mu = posterior_mu_dist(pars)
post_samp = posterior_rvs(pars, npostsamp)


x = np.linspace(0, 10, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_mu.pdf(x), line_color='blue', name='$\mbox{Оценка }\mu$'))
fig.add_trace(go.Scatter(x=[np.sum(data)/len(data), np.sum(data)/len(data)], y=[0, max(post_mu.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_mu.pdf(x))*1.05], 
                         line_color='red', mode='lines', line_dash='dash', name='Точное среднее'))
fig.update_layout(title='$\mbox{Оценка }\mu$',
                  xaxis_title='$\mu$',
                  yaxis_title='Плотность вероятности',
                  #xaxis_range=[0, 10],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_dash='solid', line_color='black', name='Точное'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='$\mbox{Апострериорное, }\mu$', nbinsx=300,
                           marker_color='green', opacity=0.2))
fig.add_trace(go.Scatter(x=x, y=clt_dist_exact.pdf(x), 
                         mode='lines', line_color='black', line_dash='dash', name='ЦПТ-распределение'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Выборочные средние', nbinsx=100,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Апостериорное распределение',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  #xaxis_range=[0, 10],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()

Распределение $\mu$ уже распределения выборочных средних.  
Это разные распределения. Распределение $\mu$ - оценка точного среднего, выборочные средние - распределение выборочных средних исходного распределения - у них больше диспресия.

Сравнение групп

In [ ]:
# p_A = 0.1
# p_B = p_A * 1.05
# nsample = 1000

# exact_dist_A = stats.bernoulli(p=p_A)
# exact_dist_B = stats.bernoulli(p=p_B)
# data_A = exact_dist_A.rvs(nsample)
# data_B = exact_dist_B.rvs(nsample)

# post_dist_A = posterior_dist_binom(ns=np.sum(data_A), ntotal=len(data_A))
# post_dist_B = posterior_dist_binom(ns=np.sum(data_B), ntotal=len(data_B))

# x = np.linspace(0, 1, 1000)
# fig = go.Figure()
# #fig.add_vline(x=exact_dist.mean(), line_dash='dash', name='Exact')
# fig.add_trace(go.Scatter(x=x, y=post_dist_A.pdf(x), line_color='black', name='A'))
# fig.add_trace(go.Scatter(x=x, y=post_dist_B.pdf(x), line_color='black', opacity=0.2, name='B'))
# fig.add_trace(go.Scatter(x=[exact_dist_A.mean(), exact_dist_A.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
#                          mode='lines', line_dash='dash', line_color='black', name='Точное A'))
# fig.add_trace(go.Scatter(x=[exact_dist_B.mean(), exact_dist_B.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
#                          mode='lines', line_dash='dash', line_color='black', opacity=0.2, name='Точное B'))
# fig.update_layout(title='Апостериорные распределения',
#                   xaxis_title='p',
#                   yaxis_title='Плотность вероятности',
#                   xaxis_range=[p_A/2, p_A*2],
#                   hovermode="x",
#                   height=500)
# fig.show()

# print(f'P(pB > pA): {prob_pb_gt_pa(post_dist_A, post_dist_B)}')

Правильно угаданные варианты

In [ ]:
# cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

# p = 0.1
# nexps = 100
# cmp['A'] = [p] * nexps
# cmp['B'] = p * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))
# cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B'] > r['A'] else 'A', axis=1)

# prob_stop = 0.95
# for i in range(nexps):
#     pA = cmp.at[i, 'A']
#     pB = cmp.at[i, 'B']
#     exact_dist_A = stats.bernoulli(p=pA)
#     exact_dist_B = stats.bernoulli(p=pB)
#     n_samp_max = 10_000_000
#     n_samp_total = 0
#     n_samp_step = 10_000
#     ns_A = 0
#     ns_B = 0
#     while n_samp_total < n_samp_max:
#         dA = exact_dist_A.rvs(n_samp_step)
#         dB = exact_dist_B.rvs(n_samp_step)
#         n_samp_total += n_samp_step
#         ns_A = ns_A + np.sum(dA)
#         ns_B = ns_B + np.sum(dB)
#         post_dist_A = posterior_dist_binom(ns=ns_A, ntotal=n_samp_total)
#         post_dist_B = posterior_dist_binom(ns=ns_B, ntotal=n_samp_total)
#         pb_gt_pa = prob_pb_gt_pa(post_dist_A, post_dist_B)
#         best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
#         if best_gr:
#             cmp.at[i, 'A_exp'] = post_dist_A.mean()
#             cmp.at[i, 'B_exp'] = post_dist_B.mean()
#             cmp.at[i, 'exp_samp_size'] = n_samp_total
#             cmp.at[i, 'best_exp'] = best_gr
#             cmp.at[i, 'p_best'] = pb_gt_pa
#             break
#     print(f'done {i}: {n_samp_total}, {best_gr}, {pb_gt_pa}')

# cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
# display(cmp.head(10))
# cor_guess = np.sum(cmp['correct'])
# print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

# Выручка на пользователя

В А/Б тестах для оценки денежного эффекта сравнивают выручку на пользователя между группами $P_{пользователи}(x)$. Для моделирования удобно выделить выручку на платящего $P_{платящие}(x)$. При конверсии в оплату $p$ распределение клиентов с ненулевой выручкой $p P_{платящие}(x)$. С вероятностью $1-p$ пользователь неплатящий, т.е. с нулевой выручкой.

$$
P_{пользователи}(x) = 
\begin{cases}
1-p, \, x = 0
\\
p P_{платящие}(x), \, x > 0
\end{cases}
$$

Оценка конверсии в оплату $p$ делалась ранее. Выручку на платящего можно моделировать логнормальным распределением [[LognormDist](https://en.wikipedia.org/wiki/Log-normal_distribution),
[SciPyLognorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html)] или распределением Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution)] по аналогии с распределением богатства [пример]. Для транзакционных сервисов, в частности маркетплейсов, лучше подходит логнормальное распределение.

Случайная величина логнормальная $X \sim Lognormal(\mu, \sigma^2)$, если логарифм распределен нормально $\ln(X) \sim Norm(\mu, \sigma^2)$. Плотность вероятности

$$
\begin{split}
f(x) & = \frac{1}{x \sigma \sqrt{2 \pi}} e^{-\tfrac{(\ln(x) - \mu)^2}{2 \sigma^2}} .
\end{split}
$$

Примеры логнормального распределения

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for sigma, loc, scale in [(1, 0, 1), (2, 0, 1), (1, 0, 2)]:
    fig.add_trace(go.Scatter(x=x, y=stats.lognorm.pdf(x, s=sigma, loc=loc, scale=scale), 
                             mode='lines', 
                             name=f's={sigma}, loc={loc}, scale={scale}'))
fig.update_layout(title='Логнормальное распределение',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  hovermode="x",
                  height=550)
fig.show()

Сопряженное априорное распределение для нормальной функции правдоподобия.  
Один параметр: $\mu$ меняется, $\sigma$ фиксировано.  
Модель с меняющимися $\mu$ и $\sigma$ см. отдельно.  
Сопряженное априорное распределение - нормальное распределение.

$$
P(data | model) = Lognorm(x | \mu, \sigma_x^2) = 
\frac{1}{x \sqrt{2 \pi \sigma_x^2}} e^{-\tfrac{(\ln x - \mu)^2}{2 \sigma_x^2}}
$$

$$
P(model) = Norm(\mu | \mu_0, \sigma_0^2) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
\prod_i^N
Lognorm(x_i | \mu, \sigma_x^2)
Norm(\mu | \mu_0, \sigma_0^2)
\\
& \propto_{\mu}
\prod_i^N
e^{-\tfrac{(\ln x_i - \mu)^2}{2 \sigma_x^2}}
e^{-\tfrac{(\mu-\mu_0)^2}{2 \sigma_0^2}} 
\\
& \propto_{\mu}
e^{-\mu^2 \left[\tfrac{N}{2 \sigma_x^2} + \tfrac{1}{2 \sigma_0^2} \right] + 
   2\mu \left[\tfrac{\mu_0}{2 \sigma_0^2} + \tfrac{1}{2 \sigma_x^2} \sum_i^N \ln x_i \right]}
\\
& \propto_{\mu}
e^{-\tfrac{(\mu - \mu_N)^2}{2 \sigma_N^2}}
= Norm(\mu | \mu_N, \sigma_N^2),
\quad
\sigma_N^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_x^2 + N \sigma_0^2},
\quad
\mu_N = \mu_0 \frac{\sigma_N^2}{\sigma_0^2} + \frac{\sigma_N^2}{\sigma_x^2} \sum_i^N \ln x_i
\end{split}
$$

Оценка параметров

In [ ]:
ConjugateLognormalParams = namedtuple('ConjugateLognormalParams', 'mu sigma sx')

def initial_params_lognormal(mu, sigma, sx):
    return ConjugateLognormalParams(mu=mu, sigma=sigma, sx=sx)

def posterior_params_lognormal(data, initial_pars):
    N = len(data)
    lnx = np.log(data)
    sigma_n_2 = (initial_pars.sigma**2 * initial_pars.sx**2) / (initial_pars.sx**2 + N * initial_pars.sigma**2)
    mu_n = initial_pars.mu * sigma_n_2 / initial_pars.sigma**2 + np.sum(lnx) * sigma_n_2 / initial_pars.sx**2    
    return ConjugateLognormalParams(mu=mu_n, sigma=np.sqrt(sigma_n_2), sx=initial_pars.sx)

def posterior_mu_dist_lognormal(params):
    return stats.norm(loc=params.mu, scale=params.sigma)

def posterior_lognormal_rvs(params, nsamp):
    mus = stats.norm.rvs(loc=params.mu, scale=params.sigma, size=nsamp)
    return stats.lognorm.rvs(s=params.sx, loc=0, scale=np.exp(mus), size=nsamp)
    
s = 1
loc = 0
scale = 2.5
nsample = 10000

exact_dist = stats.lognorm(s=s, loc=loc, scale=scale)
data = exact_dist.rvs(nsample)

# todo: avoid setting from data
lnx = np.log(data)
sx = np.std(lnx)
mu0 = lnx[0]
sigma0 = sx

pars = initial_params_lognormal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_lognormal(data[1:], pars)

print(pars)

post_mu = posterior_mu_dist_lognormal(pars)

npostsamp = 10000
post_samp = posterior_lognormal_rvs(pars, npostsamp)

xaxis_max=20
x = np.linspace(0, xaxis_max, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_color='black', name='Точное'))
fig.add_trace(go.Scatter(x=[np.sum(data)/len(data), np.sum(data)/len(data)], y=[0, max(exact_dist.pdf(x))], 
                        line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(exact_dist.pdf(x))*1.05], 
                        line_color='red', mode='lines', line_dash='dash', name='Точное среднее'))
fig.add_trace(go.Histogram(x=post_samp[post_samp < xaxis_max], histnorm='probability density', name='Апострериорное', nbinsx=100,
                          marker_color='green', opacity=0.5))
fig.update_layout(title='Апостериорное распределение',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  #xaxis_range=[0, 10],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()


# xaxis_min=0
# xaxis_max=2
# x = np.linspace(xaxis_min, xaxis_max, 10000)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=x, y=post_mu.pdf(x), line_color='blue', name='$\mbox{Оценка }\mu$'))
# # fig.add_trace(go.Scatter(x=[np.sum(data)/len(data), np.sum(data)/len(data)], y=[0, max(post_mu.pdf(x))], 
# #                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
# # fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_mu.pdf(x))*1.05], 
# #                         line_color='red', mode='lines', line_dash='dash', name='Точное среднее'))
# fig.update_layout(title='Апостериорное распределение',
#                   xaxis_title='$x$',
#                   yaxis_title='Плотность вероятности',
#                   #xaxis_range=[0, 10],
#                   barmode='overlay',
#                   hovermode="x",
#                   height=500)                  
# fig.show()

Сравнение 2 групп

Доля правильно угаданных вариантов

# Заказы на платящего

Заказы пользователя дискретная величина $P_{заказы}(n)$, $n \in 1, 2, \dots$. Можно ожидать лог-нормальное или степенное распределение - Парето, распределение Ципфа [[ZipfDist](https://en.wikipedia.org/wiki/Zipf%27s_law#Formal_definition)] $P(n) \propto n^{-s}$. Модель с меньшими ограничениями - свои вероятности $p_i$ под каждое количество заказов. Пусть масимальное количество заказов $N$ ограничено, $n_i$ - количество пользователей с $i=1, 2, \dots, N$ заказами, $p_i$ - вероятность сделать $i$ заказов. Функция правдоподобия задается мультиномиальным распределением [[MultiDist](https://en.wikipedia.org/wiki/Multinomial_distribution)]

$$
P(data|model) = P(n_1, \dots, n_N) = \frac{(n_1 + \dots + n_N)!}{n_{1}! \dots n_{N}!} p_{1}^{n_{1}} \dots p_{N}^{n_{N}} .
$$

Распределение Дирихле [[DrDist](https://en.wikipedia.org/wiki/Dirichlet_distribution)] будет сопряженным априорным распределением

$$
P(model) = 
Dir \left( p_{1}, \dots, p_{N}; \alpha_{1}, \dots, \alpha_{N} \right) = 
\dfrac{1}{B( \alpha_{1}, \dots, \alpha_{N} )} \prod_{i=1}^{N} p_{i}^{\alpha_{i}-1},
\qquad
\sum_{i=1}^{N} p_i = 1,
\qquad
p_i \in [0, 1], 
\qquad
B(\alpha_{1}, \dots, \alpha_{N}) = 
\frac{\prod \limits_{i=1}^{N} \Gamma( \alpha_{i} )}
{\Gamma \left( \sum \limits_{i=1}^{N} \alpha_{i} \right)} .
$$

Апостериорное распределение 

$$
\begin{split}
P(model | data) 
& \propto
\frac{(n_1 + \dots + n_N)!}{n_{1}! \dots n_{N}!} p_{1}^{n_{1}} \dots p_{N}^{n_{N}}
\dfrac{1}{B(\alpha_{1}, \dots ,\alpha_{N})} \prod _{i=1}^{N} p_{i}^{\alpha_{i}-1}
\\
& \propto
\prod_{i=1}^{N} p_{i}^{n_{i} + \alpha_{i} - 1}
\\
& =
Dir \left( p_{1}, \dots, p_{N}; n_1 + \alpha_{1}, \dots, n_N + \alpha_{N} \right)
\end{split}
$$

Маржинальными распределениями для каждого $p_i$ будут бета-распределения

$$
\begin{split}
f(p_i) = 
Beta( p_i; \alpha_i, \alpha_0 - \alpha_i ),
\quad
\alpha_0 = \sum_{i=1}^{N} \alpha_i .
\end{split}
$$

Оценка параметров

In [ ]:
def initial_params_dr(N):
    return np.ones(N)

def posterior_params_dr(data, initial_pars):
    u, c = np.unique(data, return_counts=True)
    post_pars = np.copy(initial_pars)
    for k, v in zip(u, c):
        post_pars[k - 1] = post_pars[k - 1] + v
    return post_pars

def posterior_nords_rvs(params, nsamp):
    nords = np.empty(nsamp)
    probs = stats.dirichlet.rvs(alpha=params, size=nsamp)
    for i, p in enumerate(probs):
        nords[i] = np.argmax(stats.multinomial.rvs(n=1, p=p)) + 1
    return nords

def posterior_marginal_dist_pi(i, params):
    imin = 1
    p_i = stats.beta(a=params[i - imin], b=np.sum(params) - params[i - imin])
    return p_i

def posterior_pi_mean_95pdi(i, params):
    p = posterior_marginal_dist_pi(i, params)
    m = p.mean()
    lower = p.ppf(0.025)
    upper = p.ppf(0.975)
    return m, lower, upper

def posterior_nords_mean(params):
    ex = 0
    for i in range(1, len(params)+1):
        m = posterior_marginal_dist_pi(i, params).mean()
        ex += i * m
    return ex

Nmax = 30
s = 1.5
nsample = 1000

exact_dist = stats.zipfian(a=s, n=Nmax)
data = exact_dist.rvs(nsample)

pars = initial_params_dr(Nmax)
pars = posterior_params_dr(data, pars)

post_samp = posterior_nords_rvs(pars, 100000)

pi = [posterior_pi_mean_95pdi(i, pars) for i in range(1, Nmax+1)]

x = np.arange(1, Nmax+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pmf(x), name='Точное распределение Ципфа'))
fig.add_trace(go.Histogram(x=data, histnorm='probability', name='Сэмпл', nbinsx=round(Nmax*2)))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability', name='Апостериорное распределение', opacity=0.5, nbinsx=round(Nmax*2)))
fig.add_trace(go.Scatter(x=x, 
                         y=[p[0] for p in pi],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi], arrayminus=[p[0] - p[1] for p in pi]), 
                         name='$\mbox{Оценки } p_i$',
                         mode='markers'))
fig.update_layout(title='Заказы на платящего',
                  xaxis_title='$x$',
                  yaxis_title='Вероятность',
                  xaxis_range=[0, Nmax+1],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()

Сравнение двух групп

In [ ]:
def prob_pb_gt_pa_postsamp(post_samp_a, post_samp_b):
    if len(post_samp_a) != len(post_samp_b):
        return None
    b_gt_a = np.sum(post_samp_b > post_samp_a)
    return b_gt_a / len(post_samp_a)

Nmax = 30
s_a = 2.7
s_b = s_a * 1.03
nsample = 3000

exact_dist_a = stats.zipfian(a=s_a, n=Nmax)
exact_dist_b = stats.zipfian(a=s_b, n=Nmax)
data_a = exact_dist_a.rvs(nsample)
data_b = exact_dist_b.rvs(nsample)

pars_a = initial_params_dr(Nmax)
pars_a = posterior_params_dr(data_a, pars_a)
pars_b = initial_params_dr(Nmax)
pars_b = posterior_params_dr(data_b, pars_b)

post_samp_len = 100000
post_samp_a = posterior_nords_rvs(pars_a, post_samp_len)
post_samp_b = posterior_nords_rvs(pars_b, post_samp_len)

pi_a = [posterior_pi_mean_95pdi(i, pars) for i in range(1, Nmax+1)]

x = np.arange(1, Nmax+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pmf(x), name='Точное распределение A'))
fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pmf(x), name='Точное распределение A'))
fig.add_vline(x=exact_dist_a.mean())
fig.add_vline(x=exact_dist_b.mean())
fig.add_trace(go.Histogram(x=post_samp_a, histnorm='probability', name='Апостериорное A', opacity=0.5, nbinsx=round(Nmax*2)))
fig.add_trace(go.Histogram(x=post_samp_b, histnorm='probability', name='Апостериорное B', opacity=0.5, nbinsx=round(Nmax*2)))
# fig.add_trace(go.Scatter(x=x, 
#                          y=[p[0] for p in pi],
#                          error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi], arrayminus=[p[0] - p[1] for p in pi]), 
#                          name='$\mbox{Оценки } p_i$',
#                          mode='markers'))
fig.update_layout(title='Заказы на платящего',
                  xaxis_title='$x$',
                  yaxis_title='Вероятность',
                  xaxis_range=[0, Nmax+1],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()

print(f'E[n_ords] A:, {posterior_nords_mean(pars_a)}, B: {posterior_nords_mean(pars_b)}')
print(f'P(pB > pA): {prob_pb_gt_pa_postsamp(post_samp_a, post_samp_b)}')

Доля правильно угаданных вариантов

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

s = 2.7
Nmax=30
nexps = 100
cmp['A'] = [s] * nexps
cmp['B'] = s * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B'] < r['A'] else 'A', axis=1)

prob_stop = 0.90
for i in range(nexps):
    s_a = cmp.at[i, 'A']
    s_b = cmp.at[i, 'B']
    exact_dist_a = stats.zipfian(a=s_a, n=Nmax)
    exact_dist_b = stats.zipfian(a=s_b, n=Nmax)
    n_samp_max = 200000
    n_samp_total = 0
    n_samp_step = 10000
    pars_a = initial_params_dr(Nmax)
    pars_b = initial_params_dr(Nmax)
    while n_samp_total < n_samp_max:
        data_a = exact_dist_a.rvs(n_samp_step)
        data_b = exact_dist_b.rvs(n_samp_step)
        n_samp_total += n_samp_step
        pars_a = posterior_params_dr(data_a, pars_a)
        pars_b = posterior_params_dr(data_b, pars_b)
        post_samp_len = 10000
        post_samp_a = posterior_nords_rvs(pars_a, post_samp_len)
        post_samp_b = posterior_nords_rvs(pars_b, post_samp_len)
        pb_gt_pa = prob_pb_gt_pa_postsamp(post_samp_a, post_samp_b)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_dist_A.mean()
            cmp.at[i, 'B_exp'] = post_dist_B.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: {n_samp_total}, {best_gr}, {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

# Средний чек

# Заключение

# Ссылки

[BinomDist] - [Binomial Distribution](https://en.wikipedia.org/wiki/Binomial_distribution), *Wikipedia.*  
[SciPyBinom] - [scipy.stats.binom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html), *SciPy Reference.*   
[BetaDist] - [Beta Distribution](https://en.wikipedia.org/wiki/Beta_distribution), *Wikipedia.*     
[SciPyBeta] - [scipy.stats.beta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html), *SciPy Reference.*   

Пациент приходит ко врачу. Симптомы - температура.   
Врач делает предположения на счет болезней - простуда, пневмония и т.д.  
Врач прикидывает, при каких болезнях эти симптомы вероятны.  
Дальше с учетом своего опыта и распространенности болезней ставит диагноз.  

Можно записать рассуждения количественно.
Для диагноза интересует $P(болезнь | симптомы)$.  
Вначале выбирается список возможных болезней и для каждой вычисляется $P(симптомы | болезнь)$.  
Распространенность болезней в регионе и опыт врача учитывает $P(болезнь)$.  

$$
P(болезнь | симптомы) \approx P(симптомы | болезнь) P(болезнь)
$$

На этом примере видны компоненты байесовского моделирования.  

Let’s walk through an example of using the **Bayesian approach** to guess what’s inside a present based on the **size of the box**.

### Setting:
You receive a box as a gift, but you don’t know what’s inside. Based on the **size of the box**, you want to update your belief about what the present might be. You have a few reasonable guesses, and you’ll use **Bayes’ Theorem** to compute the posterior probabilities of each possible item given the size of the box.

### Hypotheses (Possible Contents of the Box):
You consider three possible items that could be inside the box:

- \( H_1 \): A **book**.
- \( H_2 \): A **pair of shoes**.
- \( H_3 \): A **laptop**.

### Prior Probabilities:
Before looking at the size of the box, you have some prior beliefs about what the gift could be based on your knowledge of the person who gave you the gift. For example:

- \( P(H_1) = 0.3 \) (They often give books as gifts).
- \( P(H_2) = 0.5 \) (They’ve given shoes before).
- \( P(H_3) = 0.2 \) (They once mentioned buying you a laptop).

### Likelihoods (Based on the Size of the Box):
Now, you observe the **size of the box**, and you know how likely it is for each possible item to fit into a box of this size. Suppose the box is medium-sized. Based on that, you assign the following likelihoods:

- \( P(\text{Medium box} \mid H_1) = 0.8 \) (Books often come in medium-sized boxes).
- \( P(\text{Medium box} \mid H_2) = 0.6 \) (Shoes could fit in a medium-sized box, but not always).
- \( P(\text{Medium box} \mid H_3) = 0.3 \) (Laptops usually come in larger boxes, so this is less likely).

### Evidence (Total Probability of Observing the Box Size):
The total probability of receiving a **medium-sized box** is calculated by summing over all the possible contents:
\[
P(\text{Medium box}) = P(\text{Medium box} \mid H_1)P(H_1) + P(\text{Medium box} \mid H_2)P(H_2) + P(\text{Medium box} \mid H_3)P(H_3)
\]
\[
P(\text{Medium box}) = (0.8 \times 0.3) + (0.6 \times 0.5) + (0.3 \times 0.2) = 0.24 + 0.3 + 0.06 = 0.6
\]

### Applying Bayes’ Theorem:
Now, you can compute the **posterior probability** for each possible item using **Bayes’ Theorem**:

1. **Posterior for a Book**:
   \[
   P(H_1 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_1) P(H_1)}{P(\text{Medium box})} = \frac{0.8 \times 0.3}{0.6} = \frac{0.24}{0.6} = 0.4
   \]

2. **Posterior for Shoes**:
   \[
   P(H_2 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_2) P(H_2)}{P(\text{Medium box})} = \frac{0.6 \times 0.5}{0.6} = \frac{0.3}{0.6} = 0.5
   \]

3. **Posterior for a Laptop**:
   \[
   P(H_3 \mid \text{Medium box}) = \frac{P(\text{Medium box} \mid H_3) P(H_3)}{P(\text{Medium box})} = \frac{0.3 \times 0.2}{0.6} = \frac{0.06}{0.6} = 0.1
   \]

### Conclusion:
Based on the size of the box and your priors, you now have updated probabilities for what might be inside the box:
- \( P(\text{Book} \mid \text{Medium box}) = 0.4 \),
- \( P(\text{Shoes} \mid \text{Medium box}) = 0.5 \),
- \( P(\text{Laptop} \mid \text{Medium box}) = 0.1 \).

The most likely item is a **pair of shoes** (50% chance), followed by a **book** (40% chance). A **laptop** is the least likely (10% chance).

Аномалия в дашборде. Как выглядит процесс расследования причин?  
Вначале нужно сформулировать возможные предположения: баг в недавнем релизе, не отработали ETL-скрипты, ... .
С учетом прошлого опыта выбрать наиболее вероятное.  
Проверить.  

Можно записать рассуждения количественно.
Интересует $P(изменения | аномалия)$.  
Вначале выбирается список возможных причин и для каждой вычисляется $P(аномалия | изменения)$.  
Прошлый опыт учитывает $P(изменения)$.  

$$
P(изменения | аномалия) \approx P(аномалия | изменения) P(изменения)
$$

На этом примере видны компоненты байесовского моделирования. 

* В мешке лежат черные и белые шары, всего 1000 штук. Для оценки количества черных шаров вынимают шар, записывают цвет, возвращают обратно. За 20 повторов вынули 5 черных шаров. Сколько может быть черных шаров в мешке и каковы вероятности этих значений?

1)

При количестве черных шаров $N_B$ вероятность достать черный шар $p_B = N_B / N$.  
Вероятность вынуть 5 черных шаров в 20 экспериментах задается биномиальным распределением.

$$
P(5, 20 | N_B) = Binom(5, 20) = C^5_{20} p_B^5 p_W^{15}
$$

Возможные значения $N_B$ от 1 (мог попадаться один и тот же черный шар) до 999 (есть как минимум 1 белый).

Интересует

$$
P(N_B | 5, 20) = \frac{ P(5,20 | N_B) P(N_B)}{P(5,20)} = \frac{ P(5,20 | N_B) P(N_B)}{\sum \limits_{N_B=1}^{999} P(5,20 | N_B) P(N_B)}
$$

В знаменателе нормировочный коэффициент.

Значения $P(N_B)$ равновероятны $P(N_B) = 1/999$.

$$
P(N_B | 5, 20) 
= \frac{ P(5,20 | N_B)}{\sum \limits_{N_B=1}^{999} P(5,20 | N_B)} 
= \frac{C^5_{20} p_B^5 p_W^{15}}{\sum \limits_{N_B=1}^{999} C^5_{20} p_B^5 p_W^{15}}
= \frac{C^5_{20} N_B^5 (N - N_B)^{15}}{\sum \limits_{N_B=1}^{999} C^5_{20} N_B^5 (N-N_B)^{15}}
$$

Лучше так:
1000 шаров, достали-вернули 5, все 5 черные. Сколько всего черных?


$$
P(5 | N_B) = Binom(5, 5) = C^5_{5} p_B^5 p_W^{0} = p_B^5
$$

$$
P(N_B | 5) = \frac{ P(5 | N_B) P(N_B)}{\sum \limits_{N_B=1}^{1000} P(5 | N_B) P(N_B)}
$$

Значения $P(N_B)$ равновероятны $P(N_B) = 1/1000$.

$$
P(N_B | 5) 
= \frac{ P(5 | N_B)}{\sum \limits_{N_B=1}^{1000} P(5 | N_B)} 
= \frac{p_B^5}{\sum \limits_{N_B=1}^{1000} p_B^5}
= \frac{N_B^5}{\sum \limits_{N_B=1}^{1000} N_B^5}
$$

$$
\mbox{Beta}(1, 1) = 
\mbox{Uniform}(p) = 
\begin{cases}
1, p \in [0, 1]
\\
0, p \not\in [0, 1]
\end{cases}
$$

По закону больших чисел [[LargeNums](https://en.wikipedia.org/wiki/Law_of_large_numbers#Borel's_law_of_large_numbers)] по мере набора данных выборка приближается к точному распределению.

In [ ]:
# размер выборки до достижения prob_stop
fig = go.Figure()
fig.add_trace(go.Histogram(x=cmp['exp_samp_size'], histnorm='probability', name='Groups Sample Size', 
                           marker_color='black', nbinsx=100))
fig.update_layout(title='Размер выборки',
                  xaxis_title='Точек в группе',
                  yaxis_title='Доля экспериментов',
                  hovermode="x",
                  height=500)
fig.show()

# Кликабельность

Кликабельность - отношение кликов к показам.
Используется для баннеров.

Таблица вида id-показы-клики


|id| показы | клики
|- | ------ | -
|a | 10     | 2
|b | 5      | 0
|...|...|...


Показы на пользователя распределены по степенному закону (Ципфа, дискретизация Парето).  
Маржинальное распределение кликов близко экспоненциальному.  
Для каждого количества показов $N$ распределение кликов $P(c|N)$ близко экспоненциальному.  


Для моделирования удобнее произведение мультиномиальных распределений.

$$
P(c, N | p_{N,c}) = Mult(N | p_1, \dots, p_{maxN}) Mult(c | p_{1,N}, \dots, p_{maxC,N})
$$

In [ ]:
import numpy as np
from scipy import stats

pa = 7.1
sa = 0.1
print(pa, '±', 2*sa)
pb = 7.4
sb = 0.15
print(pb, '±', 2*sb)

prel = (pb - pa) / pa
srel = prel * np.sqrt((sa/pa)**2 + (sb/pb)**2)

prel, srel, 2 * srel
print(f'{prel*100:.1f} ± {2*srel*100:.1f}')

pd = pb - pa
sd = np.sqrt(sa**2 + sb**2)
print(f'{pd:.2f} ± {2*sd:.2f}')
stats.norm(loc=pd, scale=sd).cdf(0), 1 - stats.norm(loc=pd, scale=sd).cdf(0)